In [101]:
from helper import *

In [96]:
df_graph = pickle.load(open('../data/random_graph/ucla_20walk_graph.pkl', 'rb'))
df_dynamics = pd.read_csv("../data/dynamics_pairs/fca_k8_ucla_20walk_dynamics.csv")
y = df_dynamics.y
base = df_dynamics.baseline_width
df_dynamics = df_dynamics.loc[:, 's1_1':'s50_20']
df = pd.read_csv("../data/dynamics_pairs/fca_k8_ucla_20walk_colored_adj.csv")

In [97]:
df_feature = pd.read_csv("../data/random_graph/ucla_20walk_graph_features.csv")

In [98]:
scale = 4 # scale down colored adj
X = pd.concat([pd.DataFrame(df_graph.T), df/scale], axis=1)

In [162]:
#r: number of dictionary elements
def dict_handcraft(df_feature, base, X, y, r):
    ind_dense = sorted(range(len(df_feature.diameter)), key=lambda i: df_feature.diameter[i], reverse=True)[:100]
    ind_sparse = sorted(range(len(df_feature.diameter)), key=lambda i: df_feature.diameter[i], reverse=False)[:100]
    ind_con = X.loc[base==True,].index.tolist()[:100]
    
    X_dense = X.loc[ind_dense,] #9
    y_dense = y[ind_dense]
    X_sparse = X.loc[ind_sparse,] #4
    y_sparse = y[ind_sparse]
    X_con = X.loc[ind_con,]
    y_con = y[ind_con]
    
    W_dense, H_dense = ALS(X=X_dense.T.values, n_components=r, n_iter=100, a0 = 0, a1 = 0, a12 = 0, 
           H_nonnegativity=True, W_nonnegativity=True, compute_recons_error=True, subsample_ratio=1)
    W_sparse, H_sparse = ALS(X=X_sparse.T.values, n_components=r, n_iter=100, a0 = 0, a1 = 0, a12 = 0, 
           H_nonnegativity=True, W_nonnegativity=True, compute_recons_error=True, subsample_ratio=1)
    W_con, H_con = ALS(X=X_con.T.values, n_components=r, n_iter=100, a0 = 0, a1 = 0, a12 = 0, 
           H_nonnegativity=True, W_nonnegativity=True, compute_recons_error=True, subsample_ratio=1)

    return np.concatenate([W_dense.T, W_sparse.T, W_con.T])

In [163]:
W = dict_handcraft(df_feature, base, X, y, r)

In [164]:
from sklearn.linear_model import LogisticRegression   
Y_data = y
under_sampler = RandomUnderSampler()
X_res, y_res = under_sampler.fit_resample(X.values, Y_data)
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, 
                                                    test_size = 0.2, 
                                                    random_state = 4, 
                                                    stratify = y_res)
logreg = LogisticRegression()  
logreg.fit(np.matmul(W, X_train.T).T, y_train)  
#Y_pred = logreg.predict(X[:2, :])  
#Y_predict = logreg.predict_proba(X[:2, :])   
score = logreg.score(np.matmul(W, X_test.T).T, y_test)  
print(score)  

0.763023493360572
